Retrieve alerts found in a {jd, ra, dec} region.

In [ ]:
# min, max for time and coordinates. Keep as small as possible
jd_region = [2459117., 2459119.]
ra_region = [ 32., 33.]
dec_region = [8., 10.]
# Alert requirements
min_rb = 0.3    # RealBogus. 0.3 is the standard cutoff value for rejecting most artifacts while keeping most real
min_det = 2.    # Requiring more than one detection makes the selection much cleaner

In [ ]:
import requests, os

In [ ]:
# This is the archive token which can be obtained from https://ampel.zeuthen.desy.de/live/dashboard/tokens
# In order to retrieve ZTF partnership alerts your token needs to have the appropriate access
token = os.environ["ARCHIVE_TOKEN"]   # I have mine stored
header = {"Authorization": "bearer "+token}

#### We design a ZTF alert query, which will be posted to DESY. In return we get a stream token, with which we will later retrieve alerts. 

In [ ]:
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'

In [ ]:
query = {
  "jd": {
    "$gt": jd_region[0],
    "$lt": jd_region[1]
  },
  "candidate": {
    "ra": {
      "$gt": ra_region[0],
      "$lt": ra_region[1]
    },
    "dec": {
      "$gt": dec_region[0],
      "$lt": dec_region[1]
    },
    "rb": {
      "$gt": min_rb
    },
    "ndethist": {
      "$gte": min_det,
    }

  }
}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
if not response.ok:
    print( 'Query creation failed.')

In [ ]:
response.json()

#### We can now read alerts from this token using the ZTFArchiveAlertLoader. It takes 5+ min for the archive server to prepare the request, so in case this fails, wait and try again. 

In [ ]:
from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader

In [ ]:
config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", "stream":response.json()['resume_token']}

In [ ]:
alertloader = ZTFArchiveAlertLoader(**config)

In [ ]:
alerts = alertloader.get_alerts()

In [ ]:
acount = 0

In [ ]:
alertlist = []

In [ ]:
for alert in alerts:
    acount += 1
    alertlist.append(alert)
    print(acount, alert['objectId'], alert['candidate']['jd'])

##### We now have a list of alerts which can be examined further, e.g. by an AMPEL filter or processing unit.

In [ ]:
print('Found {} alerts'.format(acount))